<a href="https://colab.research.google.com/github/lifei96/Gingko/blob/master/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/lifei96/Gingko.git

fatal: destination path 'Gingko' already exists and is not an empty directory.


In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import *
import csv
import random

Using TensorFlow backend.
/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
f = open('Gingko/vectorized.csv')
data_0 = list()
data_1 = list()

In [4]:
csvreader = csv.reader(f)
for row in csvreader:
    row = [float(r) for r in row]
    if row[0] == 0:
        data_0.append(row)
    else:
        data_1.append(row)
f.close()

In [5]:
# balance the data
random.shuffle(data_0)
data_0 = data_0[:len(data_1)]

In [6]:
training_set = np.array(data_0[:700] + data_1[:700])
dev_set = np.array(data_0[700:] + data_1[700:])

training_labels = training_set[:, 0].astype(int)
training_data = training_set[:, 1:].astype('float32')
dev_labels = dev_set[:, 0].astype(int)
dev_data = dev_set[:, 1:].astype('float32')

In [7]:
# normalize features
_mean = np.mean(training_data, axis=0)
_std = np.std(training_data, axis=0)
training_data = (training_data - _mean) / _std
dev_data = (dev_data - _mean) / _std

In [8]:
model = Sequential()
model.add(Dense(units=50, activation='tanh', input_dim=6))
model.add(Dense(units=50, activation='tanh'))
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [9]:
model.fit(training_data, training_labels, epochs=100, batch_size=40)

Epoch 1/100
1400/1400 [==============================] - 0s 178us/step - loss: 0.6149 - acc: 0.6957
Epoch 2/100
1400/1400 [==============================] - 0s 30us/step - loss: 0.5794 - acc: 0.7236
Epoch 3/100
1400/1400 [==============================] - 0s 28us/step - loss: 0.5610 - acc: 0.7314
Epoch 4/100
1400/1400 [==============================] - 0s 28us/step - loss: 0.5501 - acc: 0.7257
Epoch 5/100
1400/1400 [==============================] - 0s 28us/step - loss: 0.5430 - acc: 0.7257
Epoch 6/100
1400/1400 [==============================] - 0s 27us/step - loss: 0.5381 - acc: 0.7300

In [10]:
model.evaluate(dev_data, dev_labels, batch_size=40)

336/336 [==============================] - 0s 86us/step


[0.43638895522980464, 0.7827380994955698]

In [17]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier

In [13]:
clf = LinearRegression()
clf.fit(training_data, training_labels)
clf.score(dev_data, dev_labels)

0.24312829026152882

In [14]:
clf = LogisticRegression()
clf.fit(training_data, training_labels)
clf.score(dev_data, dev_labels)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7767857142857143

In [15]:
clf = AdaBoostClassifier()
clf.fit(training_data, training_labels)
clf.score(dev_data, dev_labels)

0.8541666666666666

In [35]:
clf = CatBoostClassifier(iterations=10, learning_rate=1, depth=10)
clf.fit(training_data, training_labels, verbose=False)
clf.score(dev_data, dev_labels)

0.9404761904761905

In [38]:
import lightgbm as lgb

In [47]:
num_round = 10
param = {'num_leaves':31, 'num_trees':100, 'objective':'binary'}
param['metric'] = 'binary_error'
train = lgb.Dataset(training_data, label=training_labels)
dev = lgb.Dataset(dev_data, label=dev_labels)
bst = lgb.train(param, train, num_round, valid_sets=[dev])

/usr/local/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_error: 0.119048
[2]	valid_0's binary_error: 0.122024
[3]	valid_0's binary_error: 0.122024
[4]	valid_0's binary_error: 0.0952381
[5]	valid_0's binary_error: 0.0982143
[6]	valid_0's binary_error: 0.0833333
[7]	valid_0's binary_error: 0.0892857
[8]	valid_0's binary_error: 0.0892857
[9]	valid_0's binary_error: 0.0922619
[10]	valid_0's binary_error: 0.0892857
[11]	valid_0's binary_error: 0.0892857
[12]	valid_0's binary_error: 0.0892857
[13]	valid_0's binary_error: 0.0863095
[14]	valid_0's binary_error: 0.0863095
[15]	valid_0's binary_error: 0.0863095
[16]	valid_0's binary_error: 0.0803571
[17]	valid_0's binary_error: 0.0803571
[18]	valid_0's binary_error: 0.0803571
[19]	valid_0's binary_error: 0.0863095
[20]	valid_0's binary_error: 0.0863095
[21]	valid_0's binary_error: 0.0833333
[22]	valid_0's binary_error: 0.0833333
[23]	valid_0's binary_error: 0.0833333
[24]	valid_0's binary_error: 0.0833333
[25]	valid_0's binary_error: 0.0803571
[26]	valid_0's binary_error: 0.080357